In [5]:
import cv2
import numpy as np

In [6]:
# setting
colorGreen = (0, 255, 0)
scale = 0.75
img_size = (320,320)
confThreshold = 0.5
nmsThreshold = 0.4

In [7]:
# Video input
# cap = cv2.VideoCapture(0) #camera input

# if (cap.isOpened() == False):
#     print("Error opening video stream or file")

cap = cv2.VideoCapture('/Users/kancode/Desktop/Vehicle_Registration_Detection/3e6fb758869a42c6a173e8a5af229ad1.MOV') # video file input

classFile = "./v4/coco.names"
classNames = []
with open(classFile, 'r') as f:
    classNames = [line.strip() for line in f.readlines()]

modelConfiguration = "./v4/yolov4.cfg"
modelWeights = "./v4/yolov4.weights"


In [8]:
#เขียนกรอบ
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
layersNames = net.getLayerNames()
outputNames = [layersNames[i-1] for i in net.getUnconnectedOutLayers()]


def locateObjects(outputs,img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w, h = int(det[2] * wT-200), int(det[3] * hT)
                x, y = int((det[0] * wT) - w / 2), int((det[1] * hT) - h / 2)
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confs.append(float(confidence))
    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img, (x, y), (x + w, y + h), colorGreen, 2)
        label = str(classNames[classIds[i]])
        label = '{}: {:.1f}%'.format(label, confs[i]*100)
        cv2.putText(img, label, (x, y - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, colorGreen, 2)

#main Loop
while (cap.isOpened()):
    success, img = cap.read()
    img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA) #resize not important
    blob = cv2.dnn.blobFromImage(img, 0.00392, img_size, [0, 0, 0], crop=False) #1/255 is .00392
    net.setInput(blob)
    outputs = net.forward(outputNames)
    locateObjects(outputs,img) #เขียนกรอบ
    
    cv2.imshow("Image", img)
    if cv2.waitKey(1) == 27:
        break

cap.release() # ปิดการใช้งานกล้อง
cap.destroyAllWindows() # ปิดหน้าต่างทั้งหมด


IndexError: invalid index to scalar variable.